# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [15]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = 'HPRC_PacBio_HiFi_Metadata_Submission_v0.5_WUSTL.txt'
sra_metadata = 'metadata-9061008-processed-ok.tsv'

# submission id
submission_id = 'WUSTL_HPRC_HiFi_Year1'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_WUSTL_HPRC_HiFi_Year1
Workspace storage bucket: gs://fc-888bd569-6e57-439d-b69d-943f184cf8cb/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [5]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,1519989,34896456183,34.90,46,50423,22958,19506,21728,25414,20003,22667,26976,m64043_200710_174426.ccs.bam
2,1145913,26040412682,26.04,45,50025,22724,19419,21541,25057,19889,22407,26545,m64043_200711_235708.ccs.bam
2,1543254,31912051817,31.91,44,50236,20678,17856,20119,23071,18340,20871,23876,m64043_200719_070806.ccs.bam
2,1446038,30065709075,30.07,46,49923,20791,17929,20254,23212,18430,21022,24023,m64043_200720_133355.ccs.bam
2,1635278,36348091955,36.35,46,49688,22227,19624,21774,24357,20116,22356,25110,m64043_200702_173033.ccs.bam


## Submitter metadata table

In [6]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_200710_174426.ccs.bam,HG00438,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
1,m64043_200711_235708.ccs.bam,HG00438,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
2,m64043_200713_062240.ccs.bam,HG00438,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
3,m64043_200714_124814.ccs.bam,HG00438,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
4,m64136_200710_174522.ccs.bam,HG00621,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [7]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m64043_200710_174426.ccs.bam,SRR13684374,HG00438
1,m64136_200710_174522.ccs.bam,SRR13684373,HG00621
2,m64043_200716_182902.ccs.bam,SRR13684382,HG00673
3,m64043_200702_173033.ccs.bam,SRR13684381,HG00735
4,m64136_200625_174949.ccs.bam,SRR13684380,HG00741


# Merge all dataframes

In [8]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)
# metadata contains hrpcPlus samples; ignore

(80, 3) (80, 13) (148, 10)


In [9]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_200710_174426.ccs.bam,SRR13684374,HG00438,1519989,34896456183,34.90,46,50423,22958,19506,...,26976,HG00438,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
1,m64136_200710_174522.ccs.bam,SRR13684373,HG00621,1472593,32312653240,32.31,44,50331,21942,18576,...,25863,HG00621,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
2,m64043_200716_182902.ccs.bam,SRR13684382,HG00673,1395373,28919585361,28.92,47,49665,20725,17882,...,23944,HG00673,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
3,m64043_200702_173033.ccs.bam,SRR13684381,HG00735,1635278,36348091955,36.35,46,49688,22227,19624,...,25110,HG00735,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu
4,m64136_200625_174949.ccs.bam,SRR13684380,HG00741,1268202,31305642912,31.31,45,50089,24685,21678,...,28087,HG00741,PacBio Sequel II,Megaruptor 1,SageELF,8.0.0.80529,NaN,NaN,Washington University,tgraves@wustl.edu


# Upload To Tables

In [16]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)

In [11]:
merged_df.shape

(80, 24)

In [17]:
outf

'gs://fc-888bd569-6e57-439d-b69d-943f184cf8cb/WUSTL_HPRC_HiFi_Year1_post_sra_metadata.tsv'